In [1]:
import pandas as pd
import numpy as np

df_session=pd.read_csv("/Users/kimjong-gyu/Desktop/airbnb-recruiting-new-user-bookings/sessions.csv")



In [2]:
#First we deal with the Session dataset
df_session.isnull().sum()

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

In [3]:
#we drop about Missing value about User_id
df_session=df_session[df_session.user_id.notnull()]

In [4]:
df_session.isnull().sum()

user_id                0
action             79480
action_type      1122957
action_detail    1122957
device_type            0
secs_elapsed      135483
dtype: int64

In [5]:
#Missing values about action colums
df_session[df_session.action.isnull()].head(100)

user_id action   action_type action_detail      device_type  \
45    d1mm9tcy42    NaN  message_post  message_post  Windows Desktop   
138   4grx6yxeby    NaN  message_post  message_post  Windows Desktop   
317   xwxei6hdk4    NaN  message_post  message_post      iPad Tablet   
320   xwxei6hdk4    NaN  message_post  message_post      iPad Tablet   
382   ro2stddszp    NaN  message_post  message_post      Mac Desktop   
...          ...    ...           ...           ...              ...   
8092  hjepuvkxlj    NaN  message_post  message_post      Mac Desktop   
8121  hjepuvkxlj    NaN  message_post  message_post      Mac Desktop   
8151  hjepuvkxlj    NaN  message_post  message_post      Mac Desktop   
8302  kpnmjq1ppj    NaN  message_post  message_post      Mac Desktop   
8330  guj7f7sjve    NaN  message_post  message_post  Windows Desktop   

      secs_elapsed  
45             NaN  
138        59801.0  
317            NaN  
320            0.0  
382         7192.0  
...            ...  
8092        3013.0  
8121        6846.0  
8151         978.0  
8302       50640.0  
8330           NaN  

[100 rows x 6 columns]

In [6]:
#we can count message_post 
#this is same between the number of null value of the df_session action type message_post about 
#action_type 
df_session[df_session.action.isnull()].action_type.value_counts()

message_post    79480
Name: action_type, dtype: int64

In [7]:
# we set up the action to "message" about message_post of action_type 
df_session.loc[df_session.action.isnull(), 'action']='message'

In [8]:
df_session.isnull().sum()

user_id                0
action                 0
action_type      1122957
action_detail    1122957
device_type            0
secs_elapsed      135483
dtype: int64

In [9]:
# We see the missing value about action type and action_detail

print(df_session[df_session.action_type.isnull()].action.value_counts())
print("     ")
print(df_session[df_session.action_detail.isnull()].action.value_counts())




show                      580485
similar_listings_v2       168457
lookup                    161422
campaigns                 104331
track_page_view            80949
index                      16682
localization_settings       5380
uptodate                    3329
signed_out_modal            1054
currencies                   292
update                       225
braintree_client_token       120
check                        119
widget                        75
phone_verification            16
satisfy                        9
disaster_action                6
track_activity                 6
Name: action, dtype: int64
     
show                      580485
similar_listings_v2       168457
lookup                    161422
campaigns                 104331
track_page_view            80949
index                      16682
localization_settings       5380
uptodate                    3329
signed_out_modal            1054
currencies                   292
update                       225
braintree_

In [10]:
df_session['action_detail'].unique()

array([nan, 'view_search_results', 'wishlist_content_update',
       'similar_listings', 'change_trip_characteristics', 'p3',
       'header_userpic', 'contact_host', 'message_post', '-unknown-',
       'dashboard', 'create_user', 'confirm_email_link',
       'user_profile_content_update', 'user_profile', 'pending', 'p5',
       'create_phone_numbers', 'cancellation_policies', 'user_wishlists',
       'change_contact_host_dates', 'wishlist', 'message_thread',
       'request_new_confirm_email', 'send_message', 'your_trips',
       'login_page', 'login', 'login_modal', 'toggle_archived_thread',
       'p1', 'profile_verifications', 'edit_profile', 'oauth_login',
       'post_checkout_action', 'account_notification_settings',
       'update_user_profile', 'oauth_response', 'signup_modal',
       'signup_login_page', 'at_checkpoint', 'manage_listing',
       'create_listing', 'your_listings', 'profile_references',
       'list_your_space', 'popular_wishlists', 'listing_reviews_page',
    

In [11]:
df_session['action_type'].unique()

array([nan, 'click', 'data', 'view', 'submit', 'message_post',
       '-unknown-', 'booking_request', 'partner_callback',
       'booking_response', 'modify'], dtype=object)

In [12]:
def most_common_value_by_all_users(merge_df, feature):
    new_df = pd.DataFrame(merge_df.groupby('action')[feature].value_counts())
    new_df['index_tuple'] = new_df.index
    new_df['count'] = new_df[feature]
    new_columns = ['action', feature]
    for n, col in enumerate(new_columns):
        new_df[col] = new_df.index_tuple.apply(lambda index_tuple: index_tuple[n])
        
    new_df = new_df.reset_index(drop=True)
    new_df = new_df.drop(['index_tuple'], axis=1)
    
    new_df_max = pd.DataFrame(new_df.groupby('action', as_index=False)['count'].max())
    new_df_max = new_df_max.merge(new_df, on = ['action', 'count'])
    new_df_max = new_df_max.drop('count', axis=1)
    merge_df = merge_df.merge(new_df_max, left_on = 'action', right_on = 'action', how='left')
    
    return merge_df
df_session = most_common_value_by_all_users(df_session, 'action_type')
df_session = most_common_value_by_all_users(df_session, 'action_detail')
df_session.loc[df_session.action_type_x.isnull(), 'action_type_x'] = df_session.action_type_y
df_session.loc[df_session.action_detail_x.isnull(), 'action_detail_x'] = df_session.action_detail_y
df_session['action_type'] = df_session.action_type_x
df_session['action_detail'] = df_session.action_detail_x
df_session.drop(['action_type_x', 'action_detail_x', 'action_type_y', 'action_detail_y'], axis=1, inplace=True)








In [13]:
df_session.isnull().sum()

user_id               0
action                0
device_type           0
secs_elapsed     135483
action_type      415562
action_detail    415562
dtype: int64

In [14]:
# After that, we still have a lot of many missing values,
# such as missing values in action_type and action_detail 
# where action as "similar_listings_v2", "lookup" and "track_page_view" and so on.
df_session[df_session.action_type.isnull()].action.value_counts()



similar_listings_v2       168457
lookup                    161422
track_page_view            80949
uptodate                    3329
signed_out_modal            1054
braintree_client_token       120
check                        119
widget                        75
phone_verification            16
satisfy                        9
disaster_action                6
track_activity                 6
Name: action, dtype: int64

In [15]:
# action: similiar_listing 의 action_type 은 data 고, action_detail도 similar_listings이니
# action : similar_listings_v2 도 같은 타입이고 같은 디테일이라고 생각했다.

print(df_session[df_session.action == 'similar_listings'].action_type.value_counts())
print('---')
print(df_session[df_session.action == 'similar_listings'].action_detail.value_counts())



data    363423
Name: action_type, dtype: int64
---
similar_listings    363423
Name: action_detail, dtype: int64


In [16]:
#그래서 같은걸로 채워준다
df_session.loc[df_session.action == 'similar_listings_v2', 'action_type'] = 'data'
df_session.loc[df_session.action == 'similar_listings_v2', 'action_detail'] = 'similar_listings'



In [17]:
#나머지는 다른거와 연관이 없고 유의미한 자료를 찾을수 없으므로 drop 한다.
df_session.dropna(subset=['action_type'],inplace=True)


In [18]:
df_session.isnull().sum()

user_id               0
action                0
device_type           0
secs_elapsed     135480
action_type           0
action_detail         0
dtype: int64

In [19]:
# secs_elapsed 의 null 값들은 평균값으로 채워준다. 
df_session["secs_elapsed"].describe
df_session["secs_elapsed"] = df_session.groupby("action").transform(lambda x: x.fillna(x.median()))

In [20]:
df_session.isnull().sum()

user_id          0
action           0
device_type      0
secs_elapsed     0
action_type      0
action_detail    0
dtype: int64

In [21]:
df_session.secs_elapsed.describe()

count    1.028614e+07
mean     1.972972e+04
std      8.921948e+04
min      0.000000e+00
25%      2.380000e+02
50%      1.217000e+03
75%      9.491000e+03
max      1.799977e+06
Name: secs_elapsed, dtype: float64

In [22]:
df_session = df_session.astype({'secs_elapsed': 'int'})

In [24]:
print(df_session.dtypes)

user_id          object
action           object
device_type      object
secs_elapsed      int64
action_type      object
action_detail    object
dtype: object


In [27]:
#각 웹 로그 시간
df_session.head()

user_id          action      device_type  secs_elapsed action_type  \
1  d1mm9tcy42  search_results  Windows Desktop         67753       click   
3  d1mm9tcy42  search_results  Windows Desktop         22141       click   
5  d1mm9tcy42  search_results  Windows Desktop          7703       click   
7  d1mm9tcy42     personalize  Windows Desktop           831        data   
8  d1mm9tcy42           index  Windows Desktop         20842        view   

             action_detail  
1      view_search_results  
3      view_search_results  
5      view_search_results  
7  wishlist_content_update  
8      view_search_results

In [29]:
# 각 웹 로그 시간이 하루가 넘어가는건 이상하다고 판단 
# 24 시간 이상인 데이터 제거 
df_session.loc[df_session['secs_elapsed'] >24]['secs_elapsed'].describe()

count    9.754658e+06
mean     2.080415e+04
std      9.149577e+04
min      2.500000e+01
25%      3.500000e+02
50%      1.422000e+03
75%      1.043600e+04
max      1.799977e+06
Name: secs_elapsed, dtype: float64

In [30]:
df_session.loc[df_session.secs_elapsed > 24, 'secs_elapsed'] = 24

In [31]:
df_session['secs_elapsed'].unique()

array([24, 12, 15,  2,  0, 22, 13, 21, 23, 18,  9,  8, 10,  4, 17,  5,  6,
       20, 19,  3,  1, 11, 16,  7, 14])

In [59]:
df_session.to_csv("/Users/kimjong-gyu/Desktop/airbnb-recruiting-new-user-bookings/session_pre.csv")

user_id          action      device_type  secs_elapsed action_type  \
1  d1mm9tcy42  search_results  Windows Desktop            24       click   
3  d1mm9tcy42  search_results  Windows Desktop            24       click   
5  d1mm9tcy42  search_results  Windows Desktop            24       click   
7  d1mm9tcy42     personalize  Windows Desktop            24        data   
8  d1mm9tcy42           index  Windows Desktop            24        view   

   action_detail_account_notification_settings  \
1                                            0   
3                                            0   
5                                            0   
7                                            0   
8                                            0   

   action_detail_account_payment_methods  \
1                                      0   
3                                      0   
5                                      0   
7                                      0   
8                                      0   

   action_detail_account_payout_preferences  \
1                                         0   
3                                         0   
5                                         0   
7                                         0   
8                                         0   

   action_detail_account_privacy_settings  \
1                                       0   
3                                       0   
5                                       0   
7                                       0   
8                                       0   

   action_detail_account_transaction_history  ...  \
1                                          0  ...   
3                                          0  ...   
5                                          0  ...   
7                                          0  ...   
8                                          0  ...   

   action_detail_view_resolutions  action_detail_view_search_results  \
1                               0                                  1   
3                               0                                  1   
5                               0                                  1   
7                               0                                  0   
8                               0                                  1   

   action_detail_view_security_checks  action_detail_view_user_real_names  \
1                                   0                                   0   
3                                   0                                   0   
5                                   0                                   0   
7                                   0                                   0   
8                                   0                                   0   

   action_detail_wishlist  action_detail_wishlist_content_update  \
1                       0                                      0   
3                       0                                      0   
5                       0                                      0   
7                       0                                      1   
8                       0                                      0   

   action_detail_wishlist_note  action_detail_your_listings  \
1                            0                            0   
3                            0                            0   
5                            0                            0   
7                            0                            0   
8                            0                            0   

   action_detail_your_reservations  action_detail_your_trips  
1                                0                         0  
3                                0                         0  
5                                0                         0  
7                                0                         0  
8                                0                         0  

[5 rows x 159 columns]

In [55]:
df_weight.columns

Index(['user_id', 'action', 'device_type', 'secs_elapsed', 'action_type',
       'action_detail_account_notification_settings',
       'action_detail_account_payment_methods',
       'action_detail_account_payout_preferences',
       'action_detail_account_privacy_settings',
       'action_detail_account_transaction_history',
       ...
       'action_detail_view_resolutions', 'action_detail_view_search_results',
       'action_detail_view_security_checks',
       'action_detail_view_user_real_names', 'action_detail_wishlist',
       'action_detail_wishlist_content_update', 'action_detail_wishlist_note',
       'action_detail_your_listings', 'action_detail_your_reservations',
       'action_detail_your_trips'],
      dtype='object', length=159)

user_id  action_detail_account_notification_settings  \
1  d1mm9tcy42                                            0   
3  d1mm9tcy42                                            0   
5  d1mm9tcy42                                            0   
7  d1mm9tcy42                                            0   
8  d1mm9tcy42                                            0   

   action_detail_account_payment_methods  \
1                                      0   
3                                      0   
5                                      0   
7                                      0   
8                                      0   

   action_detail_account_payout_preferences  \
1                                         0   
3                                         0   
5                                         0   
7                                         0   
8                                         0   

   action_detail_account_privacy_settings  \
1                                       0   
3                                       0   
5                                       0   
7                                       0   
8                                       0   

   action_detail_account_transaction_history  action_detail_admin_templates  \
1                                          0                              0   
3                                          0                              0   
5                                          0                              0   
7                                          0                              0   
8                                          0                              0   

   action_detail_airbnb_picks_wishlists  action_detail_alteration_field  \
1                                     0                               0   
3                                     0                               0   
5                                     0                               0   
7                                     0                               0   
8                                     0                               0   

   action_detail_alteration_request  ...  action_detail_view_resolutions  \
1                                 0  ...                               0   
3                                 0  ...                               0   
5                                 0  ...                               0   
7                                 0  ...                               0   
8                                 0  ...                               0   

   action_detail_view_search_results  action_detail_view_security_checks  \
1                                  1                                   0   
3                                  1                                   0   
5                                  1                                   0   
7                                  0                                   0   
8                                  1                                   0   

   action_detail_view_user_real_names  action_detail_wishlist  \
1                                   0                       0   
3                                   0                       0   
5                                   0                       0   
7                                   0                       0   
8                                   0                       0   

   action_detail_wishlist_content_update  action_detail_wishlist_note  \
1                                      0                            0   
3                                      0                            0   
5                                      0                            0   
7                                      1                            0   
8                                      0                            0   

   action_detail_your_listings  action_detail_your_reservations  \
1                            0                                0   
3         